# Deepseek implementstion using AG2 GroupChat

You need to install `ag2[browser_use]` for the `WebSurferAgent` to work

In [ ]:
from typing import Annotated, Any, List, Optional

import nest_asyncio
from pydantic import BaseModel

import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agents.experimental import WebSurferAgent
from autogen.tools.dependency_injection import Depends, on

nest_asyncio.apply()

## Load the llm_config

In [2]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-4o-mini"]},
)
llm_config = {"config_list": config_list}

## Subtask function and initial analiser

The initial_analiser agent will generate suquestions and use the answer question function to start internal team conversation between websurfer agent and a critic to create answers for suquestions

In [3]:
def answer_question(
    question: str,
    llm_config: dict[str, Any],
) -> str:
    websurfer = WebSurferAgent(
        llm_config=llm_config,
        name="WebSurferAgent",
        system_message=(
            "You are a web surfer agent responsible for gathering information from the web to answer the provided question.\n"
        ),
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "") == "Answer confirmed.",
        human_input_mode="NEVER",
    )

    websurfer_critic = ConversableAgent(
        name="WebSurferCritic",
        system_message="You are a critic agent responsible for evaluating the answer provided by the web surfer agent.\n",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "") == "Answer confirmed.",
        human_input_mode="NEVER",
    )

    @websurfer.register_for_execution()
    @websurfer_critic.register_for_llm(
        description="Call this method when you agree that the original question has been answered correctly."
    )
    def confirm_answer() -> str:
        return "Answer confirmed."

    websurfer_critic.register_for_execution()(websurfer.tool)

    result = websurfer_critic.initiate_chat(
        websurfer, message="Please find the answer to this question: " + question, summary_method="reflection_with_llm"
    )

    return result.summary

In [4]:
class Subquestion(BaseModel):
    question: Annotated[str, "The original question."]
    answer: Annotated[Optional[str], "The answer to the question."] = None


class Task(BaseModel):
    question: Annotated[str, "The original question."]
    subquestions: Annotated[List[Subquestion], "The subquestions that need to be answered."]

In [5]:
MAX_SUBQUESTIONS = "two"

example_task = Task(
    question="What is the capital of France?", subquestions=[Subquestion(question="What is the capital of France?")]
)

initial_analyser = ConversableAgent(
    name="InitialAnalysisAgent",
    system_message=(
        "You are an expert at breaking down complex questions into smaller, focused subquestions.\n"
        "Your task is to take any question provided and divide it into clear, actionable subquestions that can be individually answered.\n"
        "Ensure the subquestions are logical, non-redundant, and cover all key aspects of the original question.\n"
        "Avoid providing answers or interpretations—focus solely on decomposition.\n"
        "After breaking down the question into subquestions, call the 'generate_subquestions' method to answer each subquestion.\n"
        "This is an example of an argument that can be passed to the 'generate_subquestions' method:\n"
        f"{{'task': {example_task.model_dump()}}}\n"
        "Do NOT forward the initial task without breaking it down into subquestions!!!\n"
        "Keep the subquestions to a maximum of {MAX_SUBQUESTIONS} subquestions.\n"
    ),
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "") == "Summary confirmed.",
    human_input_mode="NEVER",
)


@initial_analyser.register_for_llm(
    name="generate_subquestions",
    description="Generates subquestions for a task. Keep it to a maximum of {MAX_SUBQUESTIONS} subquestions.",
)
def generate_subquestions(task: Task, llm_config: Annotated[dict[str, Any], Depends(on(llm_config))]) -> Task:
    for subquestion in task.subquestions:
        subquestion.answer = answer_question(subquestion.question, llm_config=llm_config)

    return task

## Summarizer agent

Responsible for summarizing the subquestions into a complete answer

In [6]:
summarizer_agent = ConversableAgent(
    name="SummarizerAgent",
    system_message=(
        "You are a summarizer agent responsible for summarizing the results of the subquestions provided by the initial analysis agent.\n"
        "Your task is to take the subquestions and their answers and provide a concise summary of the overall question.\n"
        "Ensure the summary is clear, coherent, and covers all key aspects of the original question.\n"
        "Avoid providing new information or interpretations—focus solely on summarization.\n"
    ),
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "") == "Summary confirmed.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

## Critic agent

Responsible for evaluating the answer provided by the summarizer agent

In [7]:
critic_agent = ConversableAgent(
    name="CriticAgent",
    system_message=(
        "You are a critic agent responsible for evaluating the summary provided by the summarizer agent.\n"
        "Your task is to assess the quality of the summary based on its coherence, relevance, and completeness.\n"
        "Provide constructive feedback on how the summary can be improved.\n"
        "If the summary is satisfactory, call the 'confirm_summary' method to end the task.\n"
    ),
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "") == "Summary confirmed.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)


@critic_agent.register_for_llm(description="Call this method to confirm the summary.")
def confirm_summary() -> str:
    return "Summary confirmed."

## User proxy agent and state transitions

To enable function calling we need to create a UserProxyAgent that will act as a proxy for the user. This agent will be responsible for calling the functions of the other agents.

In [ ]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="A proxy to excute code",
    human_input_mode="NEVER",
    code_execution_config=False,
)

user_proxy.register_for_execution()(generate_subquestions)
user_proxy.register_for_execution()(confirm_summary)

In [9]:
def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    # always start with the user
    if len(messages) <= 1:
        return initial_analyser

    # if the last message is a tool call, return the tool_execution agent
    if "tool_calls" in messages[-1]:
        return user_proxy

    if last_speaker is user_proxy:
        penultimate_speaker = groupchat.agent_by_name(name=messages[-2].get("name", ""))

        if penultimate_speaker is initial_analyser:
            return summarizer_agent

        if penultimate_speaker is critic_agent:
            return initial_analyser

    if last_speaker is summarizer_agent:
        return critic_agent

    if last_speaker is critic_agent:
        return initial_analyser

    if last_speaker is initial_analyser:
        return summarizer_agent

# Run

Run the groupchat and get the final answer by calling chat_result.summary

In [ ]:
groupchat = autogen.GroupChat(
    agents=[initial_analyser, summarizer_agent, critic_agent, user_proxy],
    messages=[],
    speaker_selection_method=state_transition,
    max_round=1000,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

chat_result = user_proxy.initiate_chat(
    manager, message="Who are the founders of AG2", summary_method="reflection_with_llm"
)

chat_result.summary